In [2]:
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.0 MB/s eta 0:00:00


In [3]:
import random
import sys
import cv2
import numpy
import glob
import os
import pydicom
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.ndimage import map_coordinates
import numpy as np


dir="D:\\data_kaggle_ndsb"
preprocessed_image="D:\\data_kaggle_ndsb\\data_preprocessed"

from pathlib import Path

def create_dir_if_not_exists(path: Path):
    path.mkdir(parents=True, exist_ok=True)


def patient_directory(patient_id, base_patient_dir=".\\data_patient_predictions\\"):
    prefix = str(patient_id).rjust(4, '0')
    patient_dir = Path(base_patient_dir)
    create_dir_if_not_exists(patient_dir)
    return patient_dir

def image_directory(patient_id):
    image_dir = patient_directory(patient_id)
    create_dir_if_not_exists(image_dir)
    return image_dir

def files(base_dir, pattern):
    return list(base_dir.glob(pattern))

def patient_files(patient_id, file_type, extension=".png"):
    directory_functions = {
        "images": patient_directory
    }

    base_dir = directory_functions.get(file_type, patient_directory)
    files_dir = base_dir(patient_id)

    prefix = str(patient_id).rjust(4, '0')
    file_path = files(base_dir, prefix + "*" + extension)
    return file_path


In [ ]:
"""data
│
├── Train
│   ├── train
│   ├── study
│   ├── sax_folder1
│   │   ├── image1.dcm
│   │   ├── ...
│   ├── sax_folder2
│   │   ├── ...
│   ...
│
├── Test
│   ├── test
│   ├── study
│   ├── sax_folder1
│   │   ├── ...
│   ├── ...
│
└── Validate
    ├── validate
    ├── study
    ├── sax_folder1
    │   ├── ...
    ├── ...
"""

In [ ]:
from pathlib import Path

class DicomWrapper:
    def __init__(self, root: Path, file_name: str):
       pass

def sax_folders(patient_ids=None):
    BASE_DIR = Path("data")
    MAIN_DIRECTORIES = ["Train", "Test", "Validate"]
    DICOM_EXTENSION = ".dcm"

    for main_dir_name in MAIN_DIRECTORIES:
        main_dir_path = BASE_DIR / main_dir_name
        for sax_folder in main_dir_path.glob("sax*"):
            for dicom_file in sax_folder.glob(f"*{DICOM_EXTENSION}"):

                patient_id = sax_folder.parent.name
                if patient_ids is not None and patient_id not in patient_ids:
                    continue

                dicom_data = DicomWrapper(sax_folder, dicom_file.name)
                yield dicom_data

dicom_files = list(sax_folders())
print(dicom_files)


In [ ]:
OUTPUT_SIZE = (256, 256)

def load_dicom_image(path):
    dicom = pydicom.dcmread(path)
    img = dicom.pixel_array
    return img, dicom

def normalize_pixel_spacing(image, metadata):

    px_spacing = metadata.PixelSpacing
    zoom_factor = [s/float(px_spacing[i]) for i, s in enumerate(image.shape)]
    normalized_img = cv2.resize(image, None, fx=zoom_factor[1], fy=zoom_factor[0], interpolation=cv2.INTER_LINEAR)
    return normalized_img

def center_crop(img, dim):
    center_x, center_y = img.shape[1] // 2, img.shape[0] // 2
    cropped = img[center_y - dim[0]//2 : center_y + dim[0]//2,
                  center_x - dim[1]//2 : center_x + dim[1]//2]
    return cropped

def random_rotation(img):
    angle = np.random.uniform(-15, 15)
    rotated = cv2.getRotationMatrix2D((img.shape[1]/2, img.shape[0]/2), angle, 1)
    rotated_img = cv2.warpAffine(img, rotated, img.shape[1::-1])
    return rotated_img

for set_name in ["Train", "Test", "Validate"]:
    for root, dirs, files in os.walk(set_name):
        for file in files:
            if file.endswith(".dcm"):
                dicom_path = os.path.join(root, file)
                image, meta = load_dicom_image(dicom_path)
                image = center_crop(image, OUTPUT_SIZE)
                image = random_rotation(image)
                image = normalize_pixel_spacing(image, meta)
                image = cv2.resize(image, OUTPUT_SIZE)
                output_path = dicom_path.replace('.dcm', '.png').replace(set_name, set_name + '_processed')
                if not os.path.exists(os.path.dirname(output_path)):
                    os.makedirs(os.path.dirname(output_path))
                cv2.imwrite(output_path, image)


In [ ]:
import pydicom as dicom
import numpy

class DicomWrapper:
    def __init__(self, file_path):
        self.raw_file = dicom.read_file(file_path)
        self.file_name = file_path.split('/')[-1]

    def get_value(self, name):
        """Utility method to retrieve the value of a given DICOM tag."""
        return self.raw_file.data_element(name).value if name in self.raw_file else None

    @property
    def pixel_array(self):
        """Returns the normalized pixel array of the DICOM image."""
        img = self.raw_file.pixel_array.astype(float) / numpy.max(self.raw_file.pixel_array)
        return img

    def dir(self):
        """List all available DICOM tags."""
        return self.raw_file.dir()

dicom_wrapper = DicomWrapper("path_to_dicom_file.dcm")
print(dicom_wrapper.get_value("Columns"))  # For example, to get "Columns".


In [ ]:
import cv2
import os

def read_img_from_folder(root_dir, img_name, label_name, scale_size=None, regress_overlay=False):
    img_path = os.path.join(root_dir, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE).astype(float)

    if regress_overlay:
        label_path = os.path.join(root_dir, label_name)
        label = cv2.imread(label_path, cv2.IMREAD_GRAYSCALE).astype(float)
    else:
        label = float(label_name)

    if scale_size:
        img = cv2.resize(img, (scale_size, scale_size), interpolation=cv2.INTER_AREA)
        if regress_overlay:
            label = cv2.resize(label, (scale_size, scale_size), interpolation=cv2.INTER_AREA)

    return img, label



Test Image


In [ ]:
with tf.keras.utils.CustomObjectScope({"dice_coef": dice_coef, "dice_loss": dice_loss}):
  model = tf.keras.models.load_model(os.path.join("/content/drive/MyDrive/files/", "model.h5"))
def preprocess_image(image_path):

    image = cv2.imread(image_path, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (256, 256))

    image = image / 255.0

    image = np.expand_dims(image, axis=0)

    return image


new_image_path = '/content/drive/MyDrive/Segmentation/0704_00000sax_08_10110_IM-6798-0008.png'
new_image = preprocess_image(new_image_path)


predicted_mask = model.predict(new_image)
save_mask='/content/drive/MyDrive/Segmentation/0704.png'

def overlay_mask_on_image(original_image, mask):
    alpha = 0.7
    overlay = original_image.copy()
    overlay[mask > 0.5] = [0, 255, 0]
    return cv2.addWeighted(overlay, alpha, original_image, 1 - alpha, 0)


plt.figure(figsize=(5, 5))
plt.subplot(1, 2, 1)
plt.imshow(new_image[0])
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(predicted_mask[0, :, :, 0], cmap='gray')
plt.title('Predicted Mask')

plt.show()


In [ ]:
predicted_mask = model.predict(new_image)

threshold_value = 0.5
binary_mask = (predicted_mask[0, :, :, 0] > threshold_value).astype(np.uint8)


new_image_8u = (new_image[0] * 255).astype(np.uint8)
overlayed_image = overlay_mask_on_image(new_image_8u, binary_mask)


plt.figure(figsize=(5, 5))
plt.subplot(1, 2, 1)
plt.imshow(new_image_8u)
plt.title('Original Image')

plt.subplot(1, 2, 2)
plt.imshow(overlayed_image)
plt.title('Overlayed Image')

plt.show()